In [ ]:
# Imports
import pygame, sys
from pygame.locals import *
import random, time

# Initializing
pygame.init()

# Setting up FPS
FPS = 60
FramePerSec = pygame.time.Clock()

# Creating colors
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Other Variables for use in the program
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
COINS_COLLECTED = 0

# Setting up Fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)

# Load background image
background = pygame.image.load("AnimatedStreet.png")

# Create a white screen
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
DISPLAYSURF.fill(WHITE)
pygame.display.set_caption("Racer Game")

# Player class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0:
            if pressed_keys[K_LEFT]:
                self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH:
            if pressed_keys[K_RIGHT]:
                self.rect.move_ip(5, 0)

# Enemy class
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)  # Fixed this line

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.bottom > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)  # Fixed this line

# Coin class
class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Coin.png")
        self.image = pygame.transform.scale(self.image, (30, 30))  # Resize coin
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)  # Fixed this line

    def move(self):
        self.rect.move_ip(0, SPEED)
        if self.rect.bottom > SCREEN_HEIGHT:
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)  # Fixed this line

# Setting up Sprites
P1 = Player()
E1 = Enemy()
C1 = Coin()

# Creating Sprites Groups
enemies = pygame.sprite.Group()
enemies.add(E1)
coins = pygame.sprite.Group()
coins.add(C1)
all_sprites = pygame.sprite.Group()
all_sprites.add(P1)
all_sprites.add(E1)
all_sprites.add(C1)

# Adding a new User event to increase speed
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

# Game Loop
while True:
    # Cycles through all events occurring
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.5
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # Draw background
    DISPLAYSURF.blit(background, (0, 0))

    # Display score and coins collected
    scores = font_small.render(f"Score: {SCORE}", True, BLACK)
    coins_text = font_small.render(f"Coins: {COINS_COLLECTED}", True, YELLOW)
    DISPLAYSURF.blit(scores, (10, 10))
    DISPLAYSURF.blit(coins_text, (10, 40))

    # Moves and Re-draws all Sprites
    for entity in all_sprites:
        entity.move()
        DISPLAYSURF.blit(entity.image, entity.rect)

    # Check for collision between Player and Enemy
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.Sound('crash.wav').play()
        time.sleep(1)
        DISPLAYSURF.fill(RED)
        DISPLAYSURF.blit(game_over, (30, 250))
        pygame.display.update()
        for entity in all_sprites:
            entity.kill()
        time.sleep(2)
        pygame.quit()
        sys.exit()

    # Check for collision between Player and Coin
    if pygame.sprite.spritecollideany(P1, coins):
        COINS_COLLECTED += 1
        C1.rect.top = 0
        C1.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)  # Fixed this line

    pygame.display.update()
    FramePerSec.tick(FPS)

In [ ]:
import pygame
import random
import time

# Initialize Pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 400

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# Create the screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Snake Game")

# Clock for controlling frame rate
clock = pygame.time.Clock()

# Snake properties
snake_block = 20
snake_speed = 15
snake_list = []
snake_length = 1

# Food properties
food_x = round(random.randrange(0, SCREEN_WIDTH - snake_block) / 20.0) * 20.0
food_y = round(random.randrange(0, SCREEN_HEIGHT - snake_block) / 20.0) * 20.0

# Score and level
score = 0
level = 1

# Font for displaying score and level
font = pygame.font.SysFont(None, 35)

# Function to display score and level
def show_score_level(score, level):
    score_text = font.render(f"Score: {score}", True, WHITE)
    level_text = font.render(f"Level: {level}", True, WHITE)
    screen.blit(score_text, (10, 10))
    screen.blit(level_text, (10, 40))

# Function to draw the snake
def draw_snake(snake_block, snake_list):
    for block in snake_list:
        pygame.draw.rect(screen, GREEN, [block[0], block[1], snake_block, snake_block])

# Function to generate food at a random position
def generate_food(snake_list):
    while True:
        food_x = round(random.randrange(0, SCREEN_WIDTH - snake_block) / 20.0) * 20.0
        food_y = round(random.randrange(0, SCREEN_HEIGHT - snake_block) / 20.0) * 20.0
        if [food_x, food_y] not in snake_list:  # Ensure food doesn't spawn on the snake
            return food_x, food_y

# Game loop
running = True
snake_x = SCREEN_WIDTH / 2
snake_y = SCREEN_HEIGHT / 2
snake_x_change = 0
snake_y_change = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                snake_x_change = -snake_block
                snake_y_change = 0
            elif event.key == pygame.K_RIGHT:
                snake_x_change = snake_block
                snake_y_change = 0
            elif event.key == pygame.K_UP:
                snake_y_change = -snake_block
                snake_x_change = 0
            elif event.key == pygame.K_DOWN:
                snake_y_change = snake_block
                snake_x_change = 0

    # Check for border collision
    if snake_x >= SCREEN_WIDTH or snake_x < 0 or snake_y >= SCREEN_HEIGHT or snake_y < 0:
        running = False

    # Update snake position
    snake_x += snake_x_change
    snake_y += snake_y_change
    screen.fill(BLACK)
    pygame.draw.rect(screen, RED, [food_x, food_y, snake_block, snake_block])
    snake_head = [snake_x, snake_y]
    snake_list.append(snake_head)
    if len(snake_list) > snake_length:
        del snake_list[0]

    # Check for self-collision
    for block in snake_list[:-1]:
        if block == snake_head:
            running = False

    # Draw snake and food
    draw_snake(snake_block, snake_list)
    show_score_level(score, level)
    pygame.display.update()

    # Check for food collision
    if snake_x == food_x and snake_y == food_y:
        food_x, food_y = generate_food(snake_list)
        snake_length += 1
        score += 1

        # Increase level and speed
        if score % 3 == 0:  # Increase level every 3 foods
            level += 1
            snake_speed += 5  # Increase speed

    clock.tick(snake_speed)

# Quit Pygame
pygame.quit()

In [ ]:
import pygame

def main():
    pygame.init()
    screen = pygame.display.set_mode((640, 480))
    clock = pygame.time.Clock()
    
    radius = 15
    mode = 'blue'  # Default color mode
    drawing_tool = 'line'  # Default drawing tool
    shape_start_pos = None  # Store the starting position for shapes (rectangle/circle)
    points = []  # Store points for freehand drawing (line/eraser)
    shapes = []  # Store shapes (rectangles and circles)
    trail_points = []  # Store points for mouse trail mode
    
    # Create a separate surface for drawing
    drawing_surface = pygame.Surface((640, 480))
    drawing_surface.fill((0, 0, 0))  # Initialize with black background
    
    while True:
        pressed = pygame.key.get_pressed()
        alt_held = pressed[pygame.K_LALT] or pressed[pygame.K_RALT]
        ctrl_held = pressed[pygame.K_LCTRL] or pressed[pygame.K_RCTRL]
        
        for event in pygame.event.get():
            # Determine if X was clicked, or Ctrl+W or Alt+F4 was used
            if event.type == pygame.QUIT:
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w and ctrl_held:
                    return
                if event.key == pygame.K_F4 and alt_held:
                    return
                if event.key == pygame.K_ESCAPE:
                    return
                
                # Determine if a letter key was pressed for color or tool selection
                if event.key == pygame.K_r:
                    mode = 'red'
                elif event.key == pygame.K_g:
                    mode = 'green'
                elif event.key == pygame.K_b:
                    mode = 'blue'
                elif event.key == pygame.K_y:
                    mode = 'yellow'
                elif event.key == pygame.K_w:
                    mode = 'white'
                elif event.key == pygame.K_e:
                    drawing_tool = 'eraser'
                elif event.key == pygame.K_l:
                    drawing_tool = 'line'
                elif event.key == pygame.K_c:
                    drawing_tool = 'circle'
                elif event.key == pygame.K_s:
                    drawing_tool = 'rectangle'
                elif event.key == pygame.K_p:
                    drawing_tool = 'trail'  # Switch to mouse trail mode
                    trail_points = []  # Clear the trail when switching to trail mode
            
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # Left click
                    if drawing_tool in ['rectangle', 'circle']:
                        shape_start_pos = event.pos  # Store the starting position for shapes
                    elif drawing_tool in ['line', 'eraser', 'trail']:
                        radius = min(200, radius + 1)  # Increase radius for line/eraser/trail
                elif event.button == 3:  # Right click
                    radius = max(1, radius - 1)  # Decrease radius
            
            if event.type == pygame.MOUSEMOTION:
                if drawing_tool == 'eraser':
                    # Erase by drawing black circles directly on the drawing surface
                    pygame.draw.circle(drawing_surface, (0, 0, 0), event.pos, radius)
                elif drawing_tool == 'line':
                    # If mouse moved, add point to list (for line mode)
                    position = event.pos
                    points = points + [position]
                    points = points[-73:]  # Limit the number of points to 73 (256 / 3.5 ≈ 73)
                elif drawing_tool == 'trail':
                    # Add the current mouse position to the trail points
                    trail_points.append(event.pos)
                    trail_points = trail_points[-100:]  # Limit the number of trail points
            
            if event.type == pygame.MOUSEBUTTONUP:
                if drawing_tool in ['rectangle', 'circle'] and shape_start_pos:
                    # Draw the shape when the mouse button is released
                    if drawing_tool == 'rectangle':
                        width = abs(event.pos[0] - shape_start_pos[0])
                        height = abs(event.pos[1] - shape_start_pos[1])
                        top_left = (min(shape_start_pos[0], event.pos[0]), min(shape_start_pos[1], event.pos[1]))
                        pygame.draw.rect(drawing_surface, get_color(mode), (top_left[0], top_left[1], width, height), radius)
                    elif drawing_tool == 'circle':
                        center = shape_start_pos
                        radius_shape = int(((event.pos[0] - shape_start_pos[0]) ** 2 + (event.pos[1] - shape_start_pos[1]) ** 2) ** 0.5)
                        pygame.draw.circle(drawing_surface, get_color(mode), center, radius_shape, radius)
                    shape_start_pos = None  # Reset the starting position
        
        # Clear the screen with black
        screen.fill((0, 0, 0))
        
        # Draw the drawing surface onto the screen
        screen.blit(drawing_surface, (0, 0))
        
        # Draw all points (for line mode)
        if drawing_tool == 'line':
            i = 0
            while i < len(points) - 1:
                drawLineBetween(drawing_surface, i, points[i], points[i + 1], radius, mode)
                i += 1
        
        # Draw trail points (for mouse trail mode)
        if drawing_tool == 'trail':
            for point in trail_points:
                pygame.draw.circle(drawing_surface, get_color(mode), point, radius)
            # Remove the oldest point to make the trail vanish over time
            if len(trail_points) > 0:
                trail_points.pop(0)
        
        pygame.display.flip()
        clock.tick(60)

def get_color(mode):
    # Return the color based on the mode
    if mode == 'red':
        return (255, 0, 0)
    elif mode == 'green':
        return (0, 255, 0)
    elif mode == 'blue':
        return (0, 0, 255)
    elif mode == 'yellow':
        return (255, 255, 0)
    elif mode == 'white':
        return (255, 255, 255)
    elif mode == 'black':
        return (0, 0, 0)
    else:
        return (0, 0, 0)

def drawLineBetween(screen, index, start, end, width, color_mode):
    # Draw a line between two points with a gradient effect
    c1 = max(0, min(255, 2 * index - 256))
    c2 = max(0, min(255, 2 * index))
    
    color = get_color(color_mode)
    
    dx = start[0] - end[0]
    dy = start[1] - end[1]
    iterations = max(abs(dx), abs(dy))
    
    for i in range(iterations):
        progress = 1.0 * i / iterations
        aprogress = 1 - progress
        x = int(aprogress * start[0] + progress * end[0])
        y = int(aprogress * start[1] + progress * end[1])
        pygame.draw.circle(screen, color, (x, y), width)

main()